Open the mask file --> slice area of interest.

Using scipy.convolve, check neighbours of mask file (in 8 directions) --> output coastal cells

## Importing packages

In [15]:
from datetime import date 
from datetime import datetime
import glob
import numpy as np
from scipy import signal
import scipy.ndimage as ndimage
from scipy.ndimage import convolve
from scipy.ndimage.measurements import label, find_objects
import xarray as xr

## Opening the mask file

In [2]:
# NE coast of Tasmania

ds = xr.open_dataset('/g/data/e14/cp3790/Charuni/static_era5.nc').sel(time=slice('1982', '2018'), longitude=slice(147.0, 149.0), latitude=slice(-40.0, -42.0))
mask = xr.where(ds.lsm==0, 0, 1)

In [3]:
mask

<xarray.DataArray 'lsm' (time: 1, latitude: 9, longitude: 9)>
array([[[0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]]])
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) datetime64[ns] 2000-01-01

k is an array of weights, with the same number of dimensions as input

In [4]:
k =([[[1, 1, 1],
    [1, 0, 1],
    [1, 1, 1]]])

input array 'mask' is convolved with 'k', mode='constant' which ensures that the input is extended by filling all values beyond the edge with the same constant value

In [5]:
c = convolve(mask, k, mode='constant')

In [6]:
c

array([[[0, 1, 2, 4, 5, 5, 3, 2, 0],
        [0, 1, 4, 6, 8, 8, 5, 3, 0],
        [2, 3, 5, 6, 8, 8, 5, 3, 0],
        [3, 5, 6, 7, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [3, 5, 5, 5, 5, 5, 3, 2, 0]]])

coastal array is 1 when the mask==1 and the sum of neighbouring cells is < 8 (which means at least one cell is neighbouring a 0 (or sea grid cell)

In [8]:
coastal = np.logical_and(mask == 1, c < 8)

In [9]:
coastal.astype(int)

<xarray.DataArray 'lsm' (time: 1, latitude: 9, longitude: 9)>
array([[[0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0],
        [1, 1, 1, 1, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]]])
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) datetime64[ns] 2000-01-01

## Anomaly datasets

In [10]:
# surface air temperature anomalies
tasa = xr.open_dataset('/g/data/e14/cp3790/Charuni/surface_air_temp_anomaly.nc').sel(time=slice('1982', '2018'), longitude=slice(147.0, 149.0), latitude=slice(-40.0, -42.0))

In [11]:
tasa

<xarray.Dataset>
Dimensions:    (latitude: 9, longitude: 9, time: 13514)
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2018-12-31
Data variables:
    anomaly    (time, latitude, longitude) float32 ...

In [12]:
def fix(ds):
    # This function makes the original data array repeatable so that it matches with the shape of the other data array.

    trial = ds
    i = 0
    while i < 13514:  # <-- Here, 13514 corresponds to the timesteps in land dataset 
        trial = xr.concat([trial, ds], 'time')
        i+=1
    trial = trial.isel(time = slice(0,13514))
    # to specify the dates/time 
    trial.coords['time'] = np.arange(date(1982,1,1).toordinal(),date(2018,12,31).toordinal()+1) 
    
    t = np.arange(date(1982,1,1).toordinal(),date(2018,12,31).toordinal()+1)
    dates = [date.fromordinal(tt.astype(int)) for tt in t]
    
    trial.coords['time'] = dates

    return trial

In [13]:
anom = tasa.anomaly
anom

<xarray.DataArray 'anomaly' (time: 13514, latitude: 9, longitude: 9)>
[1094634 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2018-12-31

In [16]:
new_coastal = fix(coastal)  # new_coastal will now have the same shape as anom 

In [22]:
coast_grids = new_coastal.where(new_coastal != 0)
coast_grids

<xarray.DataArray 'lsm' (time: 13514, latitude: 9, longitude: 9)>
array([[[nan, nan,  1., ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        ...,
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1.,  1.,  1., ...,  1., nan, nan]],

       [[nan, nan,  1., ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        ...,
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1.,  1.,  1., ...,  1., nan, nan]],

       [[nan, nan,  1., ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        ...,
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1.,  1.,  1., ...,  1., nan, nan]],

       ...,

       [[nan, nan,  1., ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        ...,
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1.,  1.,  1., ...,  1., nan, nan]],

       [[nan, nan,  1., ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        ...,
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1.,  1.,  1., ...,  1., nan, nan]],

       [[nan, nan,  1., ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        [nan, nan, nan, ...,  1., nan, nan],
        ...,
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1., nan, nan, ...,  1., nan, nan],
        [ 1.,  1.,  1., ...,  1., nan, nan]]])
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) object 1982-01-01 1982-01-02 ... 2018-12-30 2018-12-31

In [23]:
coast_sel_stacked = coast_grids.stack(latlon=('latitude', 'longitude')).dropna(dim='latlon')

In [24]:
coast_sel_stacked

<xarray.DataArray 'lsm' (time: 13514, latlon: 29)>
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])
Coordinates:
  * time       (time) object 1982-01-01 1982-01-02 ... 2018-12-30 2018-12-31
  * latlon     (latlon) MultiIndex
  - latitude   (latlon) float64 -40.0 -40.0 -40.0 -40.0 ... -42.0 -42.0 -42.0
  - longitude  (latlon) float64 147.5 147.8 148.0 148.2 ... 148.0 148.2 148.5

In [161]:
# sea surface temperature anomalies 
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/MHW-sev/mhw_severity.pc90.*.nc'))
mhw = xr.open_mfdataset(files, combine='by_coords').sel(time=slice('1982', '2018')
                                                        
ssta = mhw.ssta.sel(time=slice('1982', '2018'), lon=slice(147.0, 149.0), lat=slice(-42.0, -40.0))